# Sorting an image collection by Cloud Coverage for a small area of interest

We created this tool to help with finding cloud-free Sentinel-2 (S2) images for an area of interest (AOI). Google Earth Engine allows you to sort S2 images by an image  property called "CLOUDY_PIXEL_PERCENTAGE" which represents the percentage of clouds calculated for the entire S2 image. WHen working with small AOIs, however, this approach might filter out cloudy images which are in fact cloud-free for the specific AOI. To remedy this, we decided to create a Jupyter-Notebook, where Sentinel-2 image collection can be sorted based on the cloudiness of a specific AOI, rather than on the one of the overall image.

### Workflow

If you want to utilize this notebook, you just have to install geopandas and geemap, as well as create a google-earth-engine project, that you want to use and fill in your paramters when needed.
You will need:

- a Shapefile specifying you aoi (make sure that the shapefile is in a location that can be read from by jupyter)

- the date-range you are interested in

- optionally: the tile you want to use
  and if you want to utilize an s2cloudless dataset to calculate the cloudiness of you pictures, you can later select additional paramters for that if you want to or just stick with the default parameters.

If you run the Notebook multiple times and all the layers are all already added to the map, it may be that there is an error that states that there is too much data to display, if this is the case, please just execute the cell you are interested in again, then it should display and render the map without problems.

### Accuracy

Our model was tested on a AOI of 18 square kilometers in a riparian forest of the Natura 2000 zone Salzachauen, Austria. The results were compared with reference data of cloud-free images obained through visual evaluation. The visual interpratation showed 12 cloud-free images for the year 2020. Dates of the images can be found in Validation_cloudfree_dates.txt file. Since our model sorts images depending on their cloud  cover we filtered and desplayed only the first 12 images of the sorted collection. 9 of those images correpond to the ones identified in the refference data. In the 3 images left, 2 had presence of clouds.


## Getting started / Install packages/ GEE Authentication


If you don't have geemap package installed, uncomment the cell code bellow. Then you would need to run it only once to install all packeges with pip and to initialize GEE. However, it is recomendded to do the installation with conda, as well as to create a new working environment first. More information on how to do this can be found on the official site of geemap: https://geemap.org/installation/#install-from-pypi


In [1]:
# # 1. Install geopandas and geemap

#get_ipython().system('pip install geopandas')
#get_ipython().system('pip install geemap')

# # 2. Authenticate and intialize GEE. This needs to be done only once.

import ee

ee.Authenticate()

ee.Initialize()



In [2]:
import geemap
import geopandas as gpd
import os

## Input User Parameters


In [3]:
# # 1. SELECT YOUR OWN STUDY AREA. 
# Define the path to the shapefile containing your Area Of Interest (AOI)

shapefile_path ="./Sample_data/Sample_AOI/AOI_Salzachauen_buffer_150m_WGS84_33N.shp"

# # 2. SET TIME FRAME. 
# Choose a start date and an end date for the Sentinel-2 image collection

start_date = "2020-01-01"
end_date = "2021-01-01"

# # 3. SET SENTINEL-2 PARAMETERS. 
# If you don't know which tile to use, just comment out the line below.
# Then you can run the code once and check in the console tab what tiles there are in the image collection. 
# The Tile name is the last 6 characters of the image name, excluding the 'T' in the front, which stands for 'Tile'.
# If you enter a tile name and you got error check if the name is correct

tile = "33UUP"


### Display an interactive map containing users's defined AOI


In [4]:
# # Create a map object.
Map = geemap.Map()

# Load the AOI shapefile using GeoPandas and convert it to Earth Engine object.
aoi = geemap.shp_to_ee(shapefile_path)

# Add AOI to the map.
Map.addLayer(aoi, {}, "AOI")
Map.centerObject(aoi, 11)

# Display the map.
Map

Map(center=[47.92303544098585, 12.945715130262455], controls=(WidgetControl(options=['position', 'transparent_…

## Main code


### Filter and print S2 image collection


In [5]:
# Load the Sentinel-2 data and filter by date, bounds, and tile. 
# The collection is also sorted by images cloud cover and then each image is cliped by the aoi
col = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")\
    .filterDate(start_date, end_date)\
    .filterBounds(aoi)\
    .filter(ee.Filter.eq("MGRS_TILE", tile))\
    .sort("CLOUDY_PIXEL_PERCENTAGE")\
    .map(lambda image: image.clip(aoi))

#show collection
#col

### Sorting the collection by Cloud coverage (whole images, as Google does it)


In [6]:
## Calcualte what percentage of a Sentinel-2 image has to be cloud free in order to cover users's area of interest

#  1. Calculate the AOI area in square meters and kilometers. 1 in area(1) refers to the error margin = 1m
aoi_geometry = aoi.geometry() # Ensure the AOI is an ee.Geometry object
aoi_m2 = aoi_geometry.area(1).getInfo() # here 1 in area(1) refers to the error margin = 1m
aoi_km2 = aoi_m2/1000000 # the function area() returns results in m2 which is why we need to convert to km2
print("aoi_m2: ", aoi_m2)
print("aoi_km2: ", aoi_km2)

# 2. Maximum cloud cover to be allowed in a S2 image = S2 image area - aoi area 
# If there is more cloud cover than this, it will for sure cover also the aoi area 
s2_image_km2 = 10000 # Area of a single sentinel image (tile) cover 100 x 100 km = 10 000 km2
max_cloud_percentage = 100 - ((aoi_km2*100)/s2_image_km2)
print("max_cloud_percentage: ", max_cloud_percentage)

aoi_m2:  18132046.34387494
aoi_km2:  18.13204634387494
max_cloud_percentage:  99.81867953656125


In [7]:
# set threshold + parameters -> sort
col_sort = col.filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", max_cloud_percentage))

#show collection
#col_sort

### Cloud masking using the Sentinel-2 quality assurance band (QA60) and sorting the collection

To determine the images with the least clouds for our aoi, the Quality assurance band can be used. The `QA60`-band is a bitmask that stores cloud mask information, where `0` represents no opaque clouds being present and `1` represents opaque clouds being present.

A bitqwise shift operatrion is used to isolate the the bit and then converts it into binary format. The resulting mask is then clipped to the area of interest and afterwards the percentage of clouds withion the aoi is calculated for each image. Lastly the percentages are added as a property to the collection and the collection is sorted by said value.

The cloudsort using the `QA60`-band is quite simple, the drawback however is that it is not very flexible. It is better suited for simple operations and smaller areas of interest, where the likelihood of images of the aoi without any cloud coverage being present is greater.


In [8]:
# Convert the aoi-feature collection into a geometry, if not done already
#aoi_geometry = aoi.geometry()


def calc_cloud_coverage(image):

    # Calculate binary mask with cloud values from QA60-band
    cloud_mask = image.select("QA60").bitwiseAnd(1 << 10).neq(0)

    # Clip cloud mask to aoi, aggregate values over region -> calculate pixels in aoi
    cloud_area = cloud_mask.clip(aoi_geometry).reduceRegion(
        # Sum up cloud mask values (how many cloud-values are present)
        reducer=ee.Reducer.sum(),
        # specify aoi as area where reduction takes place
        geometry=aoi_geometry,
        # set maximum number of pixels to consider
        maxPixels=1e9
    # retrieve number of cloudy pixels within the aoi
    ).get("QA60")

    # convert object into number & calculate the area of the aoi in square meters
    total_area = ee.Number(aoi_geometry.area())

    # Get fraction of aoi covered in clouds, convert to percentage
    cloud_coverage = ee.Number(cloud_area).divide(total_area).multiply(100)

    # add cloud coverage to image as a property
    return image.set("aoi_cloud_coverage", cloud_coverage)

# Iterate over the collection to add cloud coverage
col_sort_aoi = col.map(calc_cloud_coverage)

# Sort by cloud coverage within the aoi
sorted_collection = col.sort("aoi_cloud_coverage")

#show collection
#sorted_collection


### Visualize the 5 most cloud free images from the cloud mask created with the Sentinel-2 Quality-assurance-band


In [ ]:
# Define a number of images (X) to be visualized

num_img = 5 

# Visualize the first (X) images of the sorted cloud free S2 collection

images_list = sorted_collection.toList(num_img)
for i in range(num_img):
    image = ee.Image(images_list.get(i))
    # Obtain information for the date of the image
    date = image.date().format('yyyy_MM_dd').getInfo()
    # Define visualization parameters (adjust as needed)
    vis_params = {
        "bands": ["B4", "B3", "B2"],  # True color composite
        "min": 0,
        "max": 3000,
        "gamma": 1.4
     }
     # Add the image to the map
    Map.addLayer(image, vis_params, f"i{i+1}_{date}_QA60")
    Map.centerObject(aoi, 11)

Map

For a small area of interest this approach works, as it is very likely that there are cloud free images to work with, if we want to work with a larger area of interest, we can work with s2cloudless, which works using the cloud probability band, allowing the user to manually adapt what is considered a cloud, which may be useful, as the QA60 band essentially only uses binary classifiers, which results in some values potentially being mislabelled, thin clouds or cloud shadows for example.

using s2cloudless grants greater flexibility, but also needs more fine-tuning, as the selection of the cloud-threshold, that is best suited varies depending on the cloud type and the location. This means, some trial and error to set a suitable percentage may be needed.


### Cloud masking using s2cloudless and sorting the collection

For this approach, the cloud cloud probability band of the s2cloudless-collection is used. It stores information about the occurence of clouds, similarly to the QA60 band, but with this approach more can be customized, we can set the threshold for images to be considered, as well as the size of the buffer around the detected clouds and from what threshold on dark pixels should be considered as cloud-shadows, this allows, like stated previously for more flexibility, but the parameters are dependent on the image you are using, the types of clouds apparent and the area of interests traits.

We start by setting the paramters needed and joining the two collections:


In [9]:
# Define Additonal Parameters for cloud masking
cloud_probability_threshold = 50  # Cloud probability threshold
NIR_dark_threshold = 0.15       # Near-infrared dark threshold for cloud shadow detection
cloud_proj_distance = 1            # Cloud projection distance
buffer = 50                 # buffer distance to mask small clouds

In [10]:
def get_s2_cloudless_col(aoi, start_date, end_date):
    # Dince we already filtered the s2-image collection we do not have to add itt here again, we can just call the colelction with "col"
    
    # Filter s2cloudless collection
    s2_cloudless_col = ee.ImageCollection("COPERNICUS/S2_CLOUD_PROBABILITY").filterBounds(aoi).filterDate(start_date, end_date)
    
    #Join collections based on system:index
    return ee.ImageCollection(ee.Join.saveFirst("s2cloudless").apply(**{
        "primary": col,
        "secondary": s2_cloudless_col,
        "condition": ee.Filter.equals(leftField="system:index", rightField="system:index")
    }))

#get and deisplay joined collection
s2_cloudless_joined_col = get_s2_cloudless_col(aoi, start_date, end_date)

#show collection
#s2_cloudless_joined_col

We then proceed to create a cloud mask for the area of interest, selecting the cloud probability band from each image and calculating the cloud percentage of each image.


In [11]:
def add_cloud_bands(img):
    # Get probability from s2cloudless dataset
    cloud_probability = ee.Image(img.get("s2cloudless")).select("probability")
    
    # Identify pixels
    is_cloud = cloud_probability.gt(cloud_probability_threshold).rename("clouds")
    
    # Add probability and mask as bands
    return img.addBands(ee.Image([cloud_probability, is_cloud]))

def calculate_cloud_coverage(img):
    # call function to add cloud bands to img
    img = add_cloud_bands(img)
    
    ## Clip cloud mask to aoi, aggregate values over region -> calculate pixels in aoi
    total_pixels = ee.Number(img.clip(aoi).reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=aoi.geometry(),
        scale=10,
        maxPixels=1e9
    ).values().get(0))
    
    # Calculate cloud pixels in aoi
    cloud_pixels = ee.Number(img.select("clouds").clip(aoi).reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=aoi.geometry(),
        scale=10,
        maxPixels=1e9
    ).values().get(0))
    
    # Get fraction of pixels identified as clouds, convert to percentage
    cloud_coverage = cloud_pixels.divide(total_pixels).multiply(100)
    
    # Add the cloud coverage as image property
    return img.set("cloud_coverage", cloud_coverage)

Lastly we execute the functins created earlier, to iterate over each image in the collection by cloud coverage.


In [12]:
# Iterate over collection to apply the cloud coverage calculation to each image
s2c_collection_with_coverage = s2_cloudless_joined_col.map(calculate_cloud_coverage)

# Sort collection by cloud coverage
s2c_sorted_collection = s2c_collection_with_coverage.sort("cloud_coverage")

### Visualize the 5 most cloud free images from the cloud mask created with s2cloudless


In [15]:
# Define a number of images (X) to be visualized

num_img = 5 

# Visualize the first (X) images of the sorted cloud free S2 collection
images_list = s2c_sorted_collection.toList(num_img)
for i in range(num_img):
    image = ee.Image(images_list.get(i))
    # Obtain information for the date of the image
    date = image.date().format('YYYY-MM-dd').getInfo()
    # Define visualization parameters (adjust as needed)
    vis_params = {
        "bands": ["B4", "B3", "B2"],  # True color composite
        "min": 0,
        "max": 3000,
        "gamma": 1.4
     }
     # Add the image to the map
    Map.addLayer(image, vis_params, f"i{i+1}_{date}_s2c")
    Map.centerObject(aoi, 11)

Map

Map(bottom=182717.0, center=[47.92303544098585, 12.945715130262455], controls=(WidgetControl(options=['positio…